In [33]:
'''
transform the skeleton data in NTU RGB+D dataset into the numpy arrays for a more efficient data loading
'''

#Just looking at the data present in each skeleton 
import numpy as np
import os
import sys 



load_npy_path = './test_numpy/S001C001P001R002A060.skeleton.npy'


data = np.load(load_npy_path, allow_pickle=True).item()


for d in data:
    print(d)
    print((data["file_name"]))



file_name
S001C001P001R002A060
nbodys
S001C001P001R002A060
njoints
S001C001P001R002A060
skel_body0
S001C001P001R002A060
rgb_body0
S001C001P001R002A060
depth_body0
S001C001P001R002A060
skel_body1
S001C001P001R002A060
rgb_body1
S001C001P001R002A060
depth_body1
S001C001P001R002A060


In [5]:
from scipy import interpolate
from operator import add
import math
from scipy.spatial.distance import cosine, euclidean
from scipy import spatial

from sklearn import preprocessing
# from fastdtw import fastdtw
from scipy import interpolate
from operator import add

def extract_coords(frame):
    x = np.array(frame[0::3])
    y = np.array(frame[1::3])
    return x, y

# Get euclidean distance between frame a, b
def frame_euc_dis(a, b):
    x1, y1 = extract_coords(a)
    x2, y2 = extract_coords(b)
    dist = np.sum((x1 - x2)**2 + (y1 - y2)**2)
    return dist

# Get cosine similarity between frame a, b
def frame_cos_dis(x1, y1, x2, y2):
#     x1, y1 = extract_coords(a)
#     x2, y2 = extract_coords(b)
#     a_vec = []
#     b_vec = []
    for x, y in zip(x1, y1):
        a_vec.append(x)
        a_vec.append(y)
    for x, y in zip(x2, y2):
        b_vec.append(x)
        b_vec.append(y)
    X = np.asarray([a_vec,
        b_vec], dtype=np.float)
    X_normalized = preprocessing.normalize(X, norm='l2')
    dist = cosine(X_normalized[0,:],X_normalized[1,:])
    #dist = np.dot(a_vec, b_vec)
    return dist

In [8]:
# Code for mapping Skeleton on to the RGB VIDEOS by extracting frames
# Reading the skeleton coordinates and calculating distance between them

import cv2
import numpy as np
import os
import sys 
import matplotlib.pyplot as plt

#Points have been calculated now need to find the distance between them



# pathIn = "./test_case/S001C001P001R001A001_rgb.avi"
load_npy_path = './S001C001P001R001A001.skeleton.npy'

skeleton_data = np.load(load_npy_path, allow_pickle=True).item()

count = 0
dist = 0
# # vidcap = cv2.VideoCapture(pathIn)
# vidcap.get(len(skeleton_data["nbodys"]))

frame_count = 0

connecting_joint = {
    "1": [2,13,17],
    "2": [21],
    "3": [21,4],
    "4": [4],
    "5": [21,6],
    "6": [7],
    "7": [8],
    "8": [22,23],
    "9": [21,10],
    "10": [11],
    "11": [12],
    "12": [24,25],
    "13": [14],
    "14": [15],
    "15": [16],
    "16": [16],
    "17": [18],
    "18": [19],
    "19": [20],
    "20": [20],
    "21": [21],
    "22": [23],
    "23": [23],
    "24": [25],
    "25": [25]
}
# connecting_joint = [1, 2, 21, 3, 21, 5, 6, 7, 21, 9, 10, 11, 1, 13, 14, 15, 1, 17, 18, 19, 2, 8, 8, 12, 12]

for frame_count in range(len(skeleton_data["nbodys"])):

    person_count=0
#     img2 = image
    img2 = np.zeros((1080,1920,3), np.uint8)

    img2.fill(255)
    
    for person_count in range(skeleton_data["nbodys"][0]):
        joint_count = 1
        colorR = 255
        colorG = 127
        colorB = 127
        minx = 1800
        maxx = 0
        miny = 1080
        maxy = 0
        a_vec = []
        b_vec = []
                
        for joint_count in range(1,25):
#             print(joint_count)
            rgb_body_number = "rgb_body" + str(person_count)
            connecting_joints = connecting_joint[str(joint_count)]
            
            # Calculating distance between two fromes on each joints then take mean
            x1 = int(skeleton_data[rgb_body_number][frame_count][joint_count-1][0])
            y1 = int(skeleton_data[rgb_body_number][frame_count][joint_count-1][1])
            
            if frame_count >=1 : 
                x1_frame2 = int(skeleton_data[rgb_body_number][frame_count-1][joint_count-1][0])
                y1_frame2 = int(skeleton_data[rgb_body_number][frame_count-1][joint_count-1][1])
#                 print(x1,y1)
#                 print(x1_frame2,y1_frame2)

                #Calculating euclidean distance between the current frame 
                #and the previous frame for each joint
                dist += np.sum((x1 - x1_frame2)**2 + (y1 - y1_frame2)**2)
                
                
                #Now Calculating Consine Similarity
                a_vec.append(x1)
                a_vec.append(y1)
                b_vec.append(x1_frame2)
                b_vec.append(y1_frame2)

            
    
            #Calculating points to crop the image code here
            if x1 < minx:
                minx = x1
            elif x1 > maxx:
                maxx = x1
            if y1 < miny:
                miny = y1
            elif y1 > maxy:
                maxy = y1
            #Calculating points to crop the image code here ends
    
    
            for next_joint in connecting_joints:
#                 next_joint = connecting_joint[joint_count]
#                 print(next_joint)
                x2= int(skeleton_data[rgb_body_number][frame_count][next_joint-1][0])
                y2= int(skeleton_data[rgb_body_number][frame_count][next_joint-1][1])
                
                cv2.line(img2, (x1,y1), (x2,y2), (colorB,colorG,colorR), (10))
                colorR = 1+next_joint*6
                colorG = 255-next_joint*7
                colorB = 1+next_joint*5
                
            joint_count+=1

            
        person_count+=1
        if frame_count >=1:
            print("Euclidean Distance: ",int(dist))
            cos_sim = 1 - spatial.distance.cosine(a_vec, b_vec)
            print("Cosine Similarity : ",str(cos_sim))
            

# Incase if images with distances should be displayed then uncomment this code
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     minx-=100
#     maxx+=100
#     miny-=50
#     maxy+=50
#     cv2.putText(img2,'Distance : '+str(dist),(minx+10,miny+30), font, 1,(0,0,255),2)
#     dist=0

#     img2 = img2[miny:maxy, minx:maxx]

#     cv2.imwrite("./test_case/S001C001P001R001A001/_frame%d.jpg" % count, img2)     # save frame as JPEG file
#     #print ('Read a new frame: ', success)
    count = count + 1
    
count

VECTORS!
[1036, 518] [1036, 519]
VECTORS!
[1036, 518, 1042, 444] [1036, 519, 1041, 444]
VECTORS!
[1036, 518, 1042, 444, 1048, 367] [1036, 519, 1041, 444, 1047, 367]
VECTORS!
[1036, 518, 1042, 444, 1048, 367, 1039, 329] [1036, 519, 1041, 444, 1047, 367, 1039, 328]
VECTORS!
[1036, 518, 1042, 444, 1048, 367, 1039, 329, 1008, 392] [1036, 519, 1041, 444, 1047, 367, 1039, 328, 1007, 393]
VECTORS!
[1036, 518, 1042, 444, 1048, 367, 1039, 329, 1008, 392, 1005, 445] [1036, 519, 1041, 444, 1047, 367, 1039, 328, 1007, 393, 1004, 446]
VECTORS!
[1036, 518, 1042, 444, 1048, 367, 1039, 329, 1008, 392, 1005, 445, 1012, 464] [1036, 519, 1041, 444, 1047, 367, 1039, 328, 1007, 393, 1004, 446, 1011, 465]
VECTORS!
[1036, 518, 1042, 444, 1048, 367, 1039, 329, 1008, 392, 1005, 445, 1012, 464, 1011, 472] [1036, 519, 1041, 444, 1047, 367, 1039, 328, 1007, 393, 1004, 446, 1011, 465, 1011, 472]
VECTORS!
[1036, 518, 1042, 444, 1048, 367, 1039, 329, 1008, 392, 1005, 445, 1012, 464, 1011, 472, 1078, 398] [1036, 519,

VECTORS!
[1037, 515, 1043, 441, 1049, 364, 1038, 330, 1008, 397, 1007, 459, 1022, 434, 1035, 424, 1075, 397, 1058, 441, 1046, 386, 1033, 375, 1023, 514, 1027, 598, 1037, 678, 1019, 693, 1052, 516, 1064, 601, 1067, 681, 1056, 700, 1047, 384, 1045, 417, 1043, 438] [1037, 515, 1043, 441, 1048, 364, 1039, 329, 1008, 397, 1007, 458, 1022, 434, 1038, 423, 1076, 397, 1060, 442, 1037, 371, 1022, 362, 1023, 514, 1027, 597, 1037, 677, 1042, 699, 1052, 517, 1064, 601, 1067, 680, 1057, 697, 1047, 384, 1045, 418, 1043, 435]
VECTORS!
[1037, 515, 1043, 441, 1049, 364, 1038, 330, 1008, 397, 1007, 459, 1022, 434, 1035, 424, 1075, 397, 1058, 441, 1046, 386, 1033, 375, 1023, 514, 1027, 598, 1037, 678, 1019, 693, 1052, 516, 1064, 601, 1067, 681, 1056, 700, 1047, 384, 1045, 417, 1043, 438, 1018, 363] [1037, 515, 1043, 441, 1048, 364, 1039, 329, 1008, 397, 1007, 458, 1022, 434, 1038, 423, 1076, 397, 1060, 442, 1037, 371, 1022, 362, 1023, 514, 1027, 597, 1037, 677, 1042, 699, 1052, 517, 1064, 601, 1067, 680,

VECTORS!
[1036, 520, 1046, 441, 1056, 360, 1043, 320, 1011, 393, 1004, 457, 1018, 438, 1035, 427, 1076, 393, 1052, 401, 1031, 350] [1036, 520, 1046, 441, 1057, 359, 1043, 319, 1011, 394, 1004, 458, 1014, 460, 1017, 461, 1076, 393, 1052, 401, 1031, 350]
VECTORS!
[1036, 520, 1046, 441, 1056, 360, 1043, 320, 1011, 393, 1004, 457, 1018, 438, 1035, 427, 1076, 393, 1052, 401, 1031, 350, 1029, 330] [1036, 520, 1046, 441, 1057, 359, 1043, 319, 1011, 394, 1004, 458, 1014, 460, 1017, 461, 1076, 393, 1052, 401, 1031, 350, 1028, 330]
VECTORS!
[1036, 520, 1046, 441, 1056, 360, 1043, 320, 1011, 393, 1004, 457, 1018, 438, 1035, 427, 1076, 393, 1052, 401, 1031, 350, 1029, 330, 1021, 519] [1036, 520, 1046, 441, 1057, 359, 1043, 319, 1011, 394, 1004, 458, 1014, 460, 1017, 461, 1076, 393, 1052, 401, 1031, 350, 1028, 330, 1020, 519]
VECTORS!
[1036, 520, 1046, 441, 1056, 360, 1043, 320, 1011, 393, 1004, 457, 1018, 438, 1035, 427, 1076, 393, 1052, 401, 1031, 350, 1029, 330, 1021, 519, 1028, 604] [1036, 520,

103

In [5]:
# import cv2
# import numpy as np
# import os
# import sys 


# #Points have been calculated now need to find the distance between them



# pathIn = "./test_case/S001C001P001R001A001_rgb.avi"
# load_npy_path = './test_case/S001C001P001R001A001.skeleton.npy'

# skeleton_data = np.load(load_npy_path, allow_pickle=True).item()

# count = 0

# vidcap = cv2.VideoCapture(pathIn)
# vidcap.get(len(skeleton_data["nbodys"]))

# frame_count = 0

# for frame_count in range(len(skeleton_data["nbodys"])):
#     success,image = vidcap.read()
    
# #     print(len(skeleton_data["rgb_body0"][frame_count][0]))
# #     print(image.shape)
#     person_count=0
#     for person_count in range(skeleton_data["nbodys"][0]):
#         joint_count = 1
#         img2 = image
#         img2.fill(255)
#         for joint_count in range(25):
#             rgb_body_number = "rgb_body" + str(person_count)
#             x1 = int(skeleton_data[rgb_body_number][frame_count][joint_count][0])
#             y1 = int(skeleton_data[rgb_body_number][frame_count][joint_count][1])
            
            
#             x2= int(skeleton_data[rgb_body_number][frame_count][joint_count+1][0])
#             y2= int(skeleton_data[rgb_body_number][frame_count][joint_count+1][1])
 
            
#             cv2.line(img2, (x1,y1), (x2,y2), (0,255,0), (4))
            
#             dist = np.sum((x1 - x2)**2 + (y1 - y2)**2)
            
#             joint_count+=1
#         person_count+=1

        
        
#     cv2.imwrite("./test_case/S001C001P001R001A001/_frame%d.jpg" % count, image)     # save frame as JPEG file
#     #print ('Read a new frame: ', success)
#     count = count + 1

# count